# Assignment 2: Task 1

โดย

ตุลธร วงศ์ชัย รหัสนักศึกษา 63070224

ธนภัทร ธีรรัตตัญญู รหัสนักศึกษา 63070227

> **README**:
> - 'Run all' is not recommended. 
> - Section 1 must be run before any other section.
> - Read and follow instruction of each section carefully if it is provided.


# 1. Init Project

> **Instruction**: Set `ROOT_PATH` before running this section.

## 1.1 Import Libraries and Packages

In [1]:
import os
import time
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

## 1.2 Set Root Path

* If you initialize this project for first time, run code cell R1
* Otherwise, set your own root path in code cell R2

In [10]:
# ### Code Cell R1 ###

# ROOT_PATH = os.getcwd() # Get current working directory

# # Create /data directory
# dir = os.path.join(ROOT_PATH, 'data')
# os.mkdir(dir) 

# # Create /data/article_metadata directory
# dir = os.path.join(ROOT_PATH, 'data/article_metadata')
# os.mkdir(dir)

# # Create /data/datastore directory
# dir = os.path.join(ROOT_PATH, 'data/datastore')
# os.mkdir(dir) 

# # Create /data/datastore/article_contents_by_month directory
# dir = os.path.join(ROOT_PATH, 'data/datastore/article_contents_by_month')
# os.mkdir(dir) 

# # Create /data/datastore/article_titles_by_month directory
# dir = os.path.join(ROOT_PATH, 'data/datastore/article_titles_by_month')
# os.mkdir(dir) 

# # Create /data/target directory
# dir = os.path.join(ROOT_PATH, 'data/target')
# os.mkdir(dir) 

# # Create /data/target/article_categories_by_month directory
# dir = os.path.join(ROOT_PATH, 'data/target/article_categories_by_month')
# os.mkdir(dir) 

In [2]:
### Code Cell R2 ###

# Set your own root path here
ROOT_PATH = os.getcwd()

# 2. Text Scraping from Web

## 2.1 กำหนดค่าคงที่และสร้างฟังก์ชันที่ใช้งานบ่อย

2.1.1 กำหนดค่าคงที่ BASE_URL ของ News Article Archive ตามเซลล์โค้ดที่ 2.1.1

In [4]:
### Code Cell 2.1.1 ###

BASE_URL = 'http://www.it.kmitl.ac.th/~teerapong/news_archive/'

2.1.2 สร้างฟังก์ชันสำหรับส่ง GET request ไปยัง URL ที่ต้องการ พร้อมตรวจสอบว่าการส่งนั้นสำเร็จหรือไม่ (มี status เป็น 200) ถ้าไม่สำเร็จ ให้ทำการ raise Exception ตามเซลล์โค้ดที่ 2.1.2

In [5]:
### Code Cell 2.1.2 ###

def get_request(url:str,**kwargs)->requests.models.Response:
  """
  Call GET request using requests package 
  then return response if it is successful (status 200)
  otherwise throw an exception.

  Parameters
  ----------
  url : str
    URL to request
  **kwargs
    Other parameters for requests.get

  Returns
  ----------
  requests.models.Response
    Response of the request
  """
  res = requests.get(url, **kwargs)

  # If the request is not successful, throw an exception.  
  if (res.status_code != 200):
    print(res.text)
    err = Exception('The request is not successful. ' + 
                    f'Its status code is {res.status_code}. ' + 
                    f'The URL of request is {res.url}')
    raise err
  else:
    return res

## 2.2 เก็บ URL ของหน้ารวมบทความข่าวแต่ละเดือน

2.2.1 request หน้า index ของ News Article Archive และแปลงด้วย BeautifulSoup ตามเซลล์โค้ดที่ 2.2.1

In [6]:
### Code Cell 2.2.1 ###

index_url = BASE_URL + 'index.html'
res = get_request(index_url)
index_page = BeautifulSoup(res.content, 'lxml') # Parse the response

2.2.2 เนื่องจากในหน้า index จะมีรายการของเดือนที่มีทั้งหมดอยู่ใน `<li>` ซึ่งเชื่อมไปยังหน้ารวมบทความข่าวของเดือนนั้น ๆ จึงทำการดึง `<li>` tag ตามเซลล์โค้ดที่ 2.2.2.1 จากนั้นทำการดึงค่าของ `href` ของ `<a>` ที่อยู่ในแต่ละ `<li>` tag ที่ได้มา ตามเซลล์โค้ดที่ 2.2.2.2

In [7]:
### Code Cell 2.2.2.1 ###

tags = index_page.select('li')
tags

[<li>Articles — <a href="month-jan-2017.html">January</a> [118]</li>,
 <li>Articles — <a href="month-feb-2017.html">February</a> [124]</li>,
 <li>Articles — <a href="month-mar-2017.html">March</a> [116]</li>,
 <li>Articles — <a href="month-apr-2017.html">April</a> [118]</li>,
 <li>Articles — <a href="month-may-2017.html">May</a> [115]</li>,
 <li>Articles — <a href="month-jun-2017.html">June</a> [115]</li>,
 <li>Articles — <a href="month-jul-2017.html">July</a> [122]</li>,
 <li>Articles — <a href="month-aug-2017.html">August</a> [116]</li>,
 <li>Articles — <a href="month-sep-2017.html">September</a> [113]</li>,
 <li>Articles — <a href="month-oct-2017.html">October</a> [124]</li>,
 <li>Articles — <a href="month-nov-2017.html">November</a> [122]</li>,
 <li>Articles — <a href="month-dec-2017.html">December</a> [115]</li>]

In [8]:
### Code Cell 2.2.2.2 ###

month_links = list()
for tag in tags:
  link = tag.find('a')['href']
  month_links.append(link)
  print(link)

month-jan-2017.html
month-feb-2017.html
month-mar-2017.html
month-apr-2017.html
month-may-2017.html
month-jun-2017.html
month-jul-2017.html
month-aug-2017.html
month-sep-2017.html
month-oct-2017.html
month-nov-2017.html
month-dec-2017.html


## 2.3 เก็บข้อมูล metadata ของบทความข่าวในแต่ละเดือน

2.3.1 สร้างฟังก์ชันสำหรับใช้สกัดเอาข้อมูลชื่อเรื่อง หมวดหมู่ และลิงก์ ของแต่ละบทความข่าวในหน้ารวมบทความในเดือนหนึ่ง โดยจะเก็บเฉพาะบทความที่ยังเข้าถึงได้ ตามเซลล์โค้ดที่ 2.3.1

In [16]:
### Code Cell 2.3.1 ###

def scrape_month_page(month_url:str):
  """
  Scrape title, category and link of available news articles from month page.

  Parameters
  ----------
  month_url : str
    URL of month page

  Returns
  ----------
  DataFrame
    DataFrame containing title, category and link of news articles
  """

  res = get_request(month_url)
  month_page = BeautifulSoup(res.content, 'lxml') # Parse the response
  
  # Each article's title, category, and link are in tr tag,
  # so select all tr tag.
  articles = month_page.select('tbody>tr')

  article_titles = list()
  article_categories = list()
  article_links = list()

  for article in articles:

    # Category is in td tag with category class.
    category = article.select_one('td.category').string.strip()

    # Skip article if it is unavailable.
    if (category!='N/A'):
      # Title is in a tag inside td tag with title class.
      title = article.select_one('td.title>a').string
      # Link to article page is href attribute of 
      # a tag inside td tag with title class.
      link = article.select_one('td.title>a')['href']

      article_titles.append(title)
      article_categories.append(category)
      article_links.append(link)
  
  articles_df = pd.DataFrame({'title':article_titles,
                              'category':article_categories,
                              'link':article_links})  
  return articles_df

2.3.2 เก็บข้อมูล metadata ของบทความ (หัวข้อ หมวดหมู่ และลิงก์) ในแต่ละเดือน โดยใช้ฟังก์ชันจากข้อ 2.3.1 และบันทึกเป็นไฟล์ csv แยกแต่ละเดือน ตามเซลล์โค้ดที่ 2.3.2

In [17]:
### Code Cell 2.3.2 ###

month_abbr_to_number = {
  'jan':'01',
  'feb':'02',
  'mar':'03',
  'apr':'04',
  'may':'05',
  'jun':'06',
  'jul':'07',
  'aug':'08',
  'sep':'09',
  'oct':'10',
  'nov':'11',
  'dec':'12'
}

for month_link in month_links:
  
  month_url = BASE_URL + month_link
  print(f'Scraping {month_url}')
  articles_df = scrape_month_page(month_url)
  print(f'Scraped {articles_df.shape[0]} article metadata')

  month = month_link[6:9] # Get month abbriviation from month_link
  month = month_abbr_to_number[month] # Convert to month number
  year = month_link[10:14] # Get year from month_link
  file_name = f'article_metadata_{year}_{month}.csv'
  file_path = f'{ROOT_PATH}/data/article_metadata/{file_name}'
  articles_df.to_csv(file_path, index=False)
  print(f'Saved to ROOT_PATH/data/article_metadata/{file_name}')
  print('-'*10)


Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-jan-2017.html
Scraped 118 article metadata
Saved to ROOT_PATH/data/article_metadata/article_metadata_2017_01.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-feb-2017.html
Scraped 122 article metadata
Saved to ROOT_PATH/data/article_metadata/article_metadata_2017_02.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-mar-2017.html
Scraped 116 article metadata
Saved to ROOT_PATH/data/article_metadata/article_metadata_2017_03.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-apr-2017.html
Scraped 117 article metadata
Saved to ROOT_PATH/data/article_metadata/article_metadata_2017_04.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_archive/month-may-2017.html
Scraped 114 article metadata
Saved to ROOT_PATH/data/article_metadata/article_metadata_2017_05.csv
----------
Scraping http://www.it.kmitl.ac.th/~teerapong/news_arch

## 2.4 เก็บข้อมูลเนื้อความจากหน้าบทความข่าว

2.4.1 สร้างฟังก์ชันสำหรับใช้สกัดเอาข้อมูลเนื้อความจากหน้าบทความข่าว ตามเซลล์โค้ดที่ 2.4.1

In [12]:
### Code Cell 2.4.1 ###

def scrape_article_page(article_url:str):
  """
  Scrape article content from news article page with a given URL.

  Parameters
  ----------
  article_url : str
    URL of article page

  Returns
  ----------
  str
    Article content
  """

  res = get_request(article_url)
  article_page = BeautifulSoup(res.content, 'lxml') # Parse the response

  # Get content of article, which is in p tags without 'notice' class 
  # in div.main
  p_tags = article_page.select('div.main>p:not(.notice)')

  # Every article repeats its title in content section.
  # Due to inconsistent formating style, some articles repeat in b tag
  # while some repeat in p tag. In the later case, remove first item of
  # selected p_tags.
  b_tags = article_page.select('div.main>b')
  # If there is no b tag in content section, remove first item of
  # selected p_tags
  if (len(b_tags)==0):
    p_tags.pop(0)
  
  article_content = ''
  # Concatenate all paragraphs
  for p in p_tags:
    article_content += p.text + ' '

  # Replace double space with single space and 
  # remove leading and trailing whitespace
  article_content = article_content.replace('  ',' ').strip() 

  return article_content

2.4.2 อ่านไฟล์ metadata ของบทความในแต่ละเดือน จากนั้นใช้ลิงก์ที่อยู่ใน metadata ไปทำการเก็บข้อมูลเนื้อความของบทความแต่ละบทความ โดยใช้ฟังก์ชันจากข้อ 2.4.1 และบันทึกเป็นไฟล์ txt แยกตามเดือน โดยที่เก็บเนื้อความของ 1 บทความต่อบรรทัด ตามเซลล์โค้ดที่ 2.4.2.1 - 2.4.2.2

In [4]:
### Code Cell 2.4.2.1 ###

article_metadata_month_files = os.listdir(f'{ROOT_PATH}/data/article_metadata')
article_metadata_month_files

['article_metadata_2017_01.csv',
 'article_metadata_2017_02.csv',
 'article_metadata_2017_03.csv',
 'article_metadata_2017_04.csv',
 'article_metadata_2017_05.csv',
 'article_metadata_2017_06.csv',
 'article_metadata_2017_07.csv',
 'article_metadata_2017_08.csv',
 'article_metadata_2017_09.csv',
 'article_metadata_2017_10.csv',
 'article_metadata_2017_11.csv',
 'article_metadata_2017_12.csv']

In [18]:
### Code Cell 2.4.2.2 ###

count = 0 # Request count

# article_metadata_month_files is a list of metadata file names 
# from code cell 2.4.2.1
for article_metadata_month in article_metadata_month_files:

  articles_df = pd.read_csv(f'{ROOT_PATH}/data/article_metadata/{article_metadata_month}')
  print(f'Scraping {articles_df.shape[0]} articles from {article_metadata_month}')
  article_contents = ''

  # Scrape content of each article 
  for _, article in articles_df.iterrows():
    article_link = article['link']
    article_url = BASE_URL + article_link
    article_content = scrape_article_page(article_url)
    article_contents += article_content + '\n'

    # Delay for 1 sec after 25 consecutive requests
    count += 1
    if (count%25 == 0):
      time.sleep(1)

  article_contents = article_contents.rstrip('\n') # Remove last empty line

  # Save contents in a file
  year_month = article_metadata_month[17:24] # Get year and month
  file_name = f'article_contents_{year_month}.txt'
  file_path = f'{ROOT_PATH}/data/datastore/article_contents_by_month/{file_name}'
  f = open(file_path, mode='w', encoding='utf-8') # Create a file
  f.write(article_contents) # Write a file
  f.close()
  
  print(f'Saved to ROOT_PATH/data/datastore/article_contents_by_month/{file_name}')
  print('-'*10)

Scraping 118 articles from article_metadata_2017_01.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_01.txt
----------
Scraping 122 articles from article_metadata_2017_02.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_02.txt
----------
Scraping 116 articles from article_metadata_2017_03.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_03.txt
----------
Scraping 117 articles from article_metadata_2017_04.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_04.txt
----------
Scraping 114 articles from article_metadata_2017_05.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_05.txt
----------
Scraping 114 articles from article_metadata_2017_06.csv
Saved to ROOT_PATH/data/datastore/article_contents_by_month/article_contents_2017_06.txt
----------
Scraping 122 articles from article_metadata_2017_07.csv
Saved to

## 2.5 จัดเก็บข้อมูลหัวข้อและหมวดหมู่ของบทความแยกจาก metadata ของบทความ

2.5 อ่านไฟล์ metadata ของบทความในแต่ละเดือน จากนั้นดึงข้อมูลหัวข้อของบทความและบันทึกเป็นไฟล์ txt แยกตามเดือน โดยที่เก็บหัวข้อของ 1 บทความต่อบรรทัด และดึงข้อมูลหมวดหมู่ของบทความและบันทึกเป็นไฟล์ txt แยกตามเดือน โดยที่เก็บหมวดหมู่ของ 1 บทความต่อบรรทัด ตามเซลล์โค้ดที่ 2.5

In [20]:
### Code Cell 2.5 ###

# article_metadata_month_files is a list of metadata file names 
# from code cell 2.4.2.1
for article_metadata_month in article_metadata_month_files:

  articles_df = pd.read_csv(f'{ROOT_PATH}/data/article_metadata/{article_metadata_month}')

  # Get array of article titles then join with new line
  article_titles = '\n'.join(articles_df['title'].to_numpy()) 

  # Get array of article categories then join with new line
  article_categories = '\n'.join(articles_df['category'].to_numpy()) 

  
  year_month = article_metadata_month[17:24] # Get year and month

  # Save article titles in a file
  file_name = f'article_titles_{year_month}.txt'
  file_path = f'{ROOT_PATH}/data/datastore/article_titles_by_month/{file_name}'
  with open(file_path, mode='w', encoding='utf-8') as file:
    file.write(article_titles)
  
  # Save article categories in a file
  file_name = f'article_categories_{year_month}.txt'
  file_path = f'{ROOT_PATH}/data/target/article_categories_by_month/{file_name}'
  with open(file_path, mode='w', encoding='utf-8') as file:
    file.write(article_categories)
  
  print(f'Saved titles and categories of {articles_df.shape[0]} ' +
        f'articles from {article_metadata_month}')

Saved titles and categories of 118 articles from article_metadata_2017_01.csv
Saved titles and categories of 122 articles from article_metadata_2017_02.csv
Saved titles and categories of 116 articles from article_metadata_2017_03.csv
Saved titles and categories of 117 articles from article_metadata_2017_04.csv
Saved titles and categories of 114 articles from article_metadata_2017_05.csv
Saved titles and categories of 114 articles from article_metadata_2017_06.csv
Saved titles and categories of 122 articles from article_metadata_2017_07.csv
Saved titles and categories of 116 articles from article_metadata_2017_08.csv
Saved titles and categories of 112 articles from article_metadata_2017_09.csv
Saved titles and categories of 122 articles from article_metadata_2017_10.csv
Saved titles and categories of 121 articles from article_metadata_2017_11.csv
Saved titles and categories of 114 articles from article_metadata_2017_12.csv


## 2.6 รวมไฟล์ข้อมูลหัวข้อ เนื้อความ และหมวดหมู่ของบทความในทุกเดือน

2.6.1 สร้างฟังก์ชันสำหรับรวมไฟล์ ตามเซลล์โค้ดที่ 2.6.1

In [15]:
### Code Cell 2.6.1 ###

def merge_text_files(file_paths:list,out_file_path:str):
  """
  Merge text files in the same order as a given file path list
  then write an out file with a given path.

  Parameters
  ----------
  file_paths : list
    List of text file paths
  out_file_path : str
    File path to write a merged text file
  """

  data = ''

  for i in range(len(file_paths)):
    with open(file_paths[i], mode='r', encoding='utf-8') as file:
      data += file.read() # Append content of current text file to data

    # Add new line to data if it is not the last text file
    if (i < len(file_paths) - 1):
      data += '\n'

  # Write a merged text file
  with open(out_file_path, mode='w', encoding='utf-8') as file:
    file.write(data)


2.6.2 รวมไฟล์ข้อมูลหัวข้อของบทความ โดยใช้ฟังก์ชันจากข้อ 2.6.1 ตามเซลล์โค้ดที่ 2.6.2

In [16]:
### Code Cell 2.6.2 ###

# Get a list of file names in article_titles_by_month directory.
article_titles_by_month_dir = f'{ROOT_PATH}/data/datastore/article_titles_by_month'
article_titles_file_names = os.listdir(article_titles_by_month_dir)

# Add directory path to file name.
article_titles_file_paths = [
  article_titles_by_month_dir + '/' + file_name 
  for file_name in article_titles_file_names
]

article_titles_all_file_path = f'{ROOT_PATH}/data/datastore/article_titles_all.txt'

merge_text_files(article_titles_file_paths,article_titles_all_file_path)
print('Completed merging article title files')

Completed merging article title files


2.6.3 รวมไฟล์ข้อมูลเนื้อความของบทความ โดยใช้ฟังก์ชันจากข้อ 2.6.1 ตามเซลล์โค้ดที่ 2.6.3

In [55]:
### Code Cell 2.6.3 ###

# Get a list of file names in article_contents_by_month directory.
article_contents_by_month_dir = f'{ROOT_PATH}/data/datastore/article_contents_by_month'
article_contents_file_names = os.listdir(article_contents_by_month_dir)

# Add directory path to file name.
article_contents_file_paths = [
  article_contents_by_month_dir + '/' + file_name 
  for file_name in article_contents_file_names
]

article_contents_all_file_path = f'{ROOT_PATH}/data/datastore/article_contents_all.txt'

merge_text_files(article_contents_file_paths,article_contents_all_file_path)
print('Completed merging article content files')

Completed merging article content files


2.6.4 รวมไฟล์ข้อมูลหมวดหมู่ของบทความ โดยใช้ฟังก์ชันจากข้อ 2.6.1 ตามเซลล์โค้ดที่ 2.6.4

In [56]:
### Code Cell 2.6.4 ###

# Get a list of file names in article_categories_by_month directory.
article_categories_by_month_dir = f'{ROOT_PATH}/data/target/article_categories_by_month'
article_categories_file_names = os.listdir(article_categories_by_month_dir)

# Add directory path to file name.
article_categories_file_paths = [
  article_categories_by_month_dir + '/' + file_name
  for file_name in article_categories_file_names
]

article_categories_all_file_path = f'{ROOT_PATH}/data/target/article_categories_all.txt'

merge_text_files(article_categories_file_paths,article_categories_all_file_path)
print('Completed merging article catagory files')

Completed merging article catagory files


## 2.7 รวมข้อมูลหัวข้อและเนื้อความของบทความเข้าด้วยกัน

2.7 อ่านไฟล์ข้อมูลหัวข้อของบทความและไฟล์ข้อมูลเนื้อความของบทความ แล้วนำหัวข้อต่อกับเนื้อความของแต่ละบทความโดยคั่นด้วยเว้นวรรค และบันทึกเป็นไฟล์ txt โดยที่เก็บหัวข้อที่ต่อด้วยเนื้อความของ 1 บทความต่อบรรทัด ตามเซลล์โค้ดที่ 2.7

In [21]:
### Code Cell 2.7  ###

# Define article title file path and article content file path to read
titles_file_path = f'{ROOT_PATH}/data/datastore/article_titles_all.txt'
contents_file_path = f'{ROOT_PATH}/data/datastore/article_contents_all.txt'

# Define file path to write
out_file_name = 'article_titles_plus_contents_all.txt'
out_file_path = f'{ROOT_PATH}/data/datastore/{out_file_name}'

# Read article title file and article content file 
# and merge them line by line into an out file.
with open(titles_file_path, mode='r', encoding='utf-8') as titles_file:
  with open(contents_file_path, mode='r', encoding='utf-8') as contents_file:
    with open(out_file_path, mode='w', encoding='utf-8') as out_file:
      titles = titles_file.readlines()
      contents = contents_file.readlines()

      titles_plus_contents = ''

      # Merge titles and contents line by line
      for title, content in zip(titles, contents):
        title_plus_content = title.rstrip('\n') + ' ' + content.rstrip('\n')
        titles_plus_contents += title_plus_content + '\n'
      
      # Remove last empty line
      titles_plus_contents = titles_plus_contents.rstrip('\n') 

      out_file.write(titles_plus_contents) # Write title plus content file

print('Completed merging article_titles_all.txt and article_contents_all.txt')

Completed merging article_titles_all.txt and article_contents_all.txt


# 3. Multi-Class Classification

> **Instruction**: Scraped data from section 2 must exist in `ROOT_PATH/data` before running this section.

## 3.1 อ่านข้อมูลเนื้อความและหมวดหมู่ของบทความที่ได้จาก section 2

3.1.1 อ่านข้อมูลเนื้อความของบทความ ตามเซลล์โค้ดที่ 3.1.1

In [18]:
### Code Cell 3.1.1  ###

article_contents = pd.read_csv(f'{ROOT_PATH}/data/datastore/article_contents_all.txt',
                               sep='\n', header=None)
article_contents.columns = ['content']
article_contents

,content
0,The sporting industry has come a long way sinc...
1,Shares in Europe's leading reinsurers and trav...
2,BT is offering customers free internet telepho...
3,Shares in UK banking group Barclays have risen...
4,England centre Olly Barkley has been passed fi...
...,...
1403,Toulouse's former Irish international Trevor B...
1404,"The trial of Bernie Ebbers, former chief execu..."
1405,Russian oil firm Yukos lied to a US court in a...
1406,Russian oil company Yukos has dropped the thre...


3.1.2 อ่านข้อมูลหมวดหมู่ของบทความ ตามเซลล์โค้ดที่ 3.1.2

In [8]:
### Code Cell 3.1.2  ###

targets = pd.read_csv(f'{ROOT_PATH}/data/target/article_categories_all.txt',
                      sep='\n', header=None)
targets.columns = ['target']
targets

,target
0,technology
1,business
2,technology
3,business
4,sport
...,...
1403,sport
1404,business
1405,business
1406,business
